### Práctica sobre Desarrollo de aplicaciones web con Bottle

Se desea ampliar la práctica 3 sobre los servicios web de la EMT creando una aplicación web que actúe a modo de capa de presentación. Para ello se va a utilizar Bottle.

Se pide crear una aplicación web que tenga las siguientes páginas:

*  Una página principal que mostrará un conjunto de varios enlaces que representan los servicios que ofrece la aplicación[1 punto]:

   * Servicio 1:Servicio de obtener la ruta optima entre dos destinos usando los servicios de la EMT
   
   * Servicio 2:Servicio de mostrar los lugares de interés entre dos destinos indicados.
   
   * Servicio 3:Servicio de mostrar las paradas de autobús más cercanas con toda la información acerca de la parada: líneas que pasan, frecuencia, ...
   
* Cuando el usuario pulsa sobre el servicio 1 se le mostrará un formulario en el que podrá introducir el nombre de la calle y número de un origen y un destino de la capital de Madrid, y cuando pulse sobre un botón de "Enviar", se le mostrará una nueva página que mostrará la descripción de la ruta. En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial. [3 puntos]

* Cuando el usuario pulsa sobre el servicio 2 se le mostrará un formulario en el que podrá introducir el nombre de la calle y número de un origen y un destino de la capital de Madrid, y cuando pulse sobre un botón de "Enviar", se le mostrará una nueva página que mostrará un listado con los lugares de interés recuperados en la ruta entre las dos calles. De cada resultado mostrará la información disponible: nombre, teléfono, calle,...En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial.[3 puntos]

* Este servicio es nuevo, y requiere el uso de otro servicio web de la EMT. En este caso, se va a utilizar el servicio GetStreet de GEO. Cuando el usuario pulsa sobre el servicio 3 se le mostrará un formulario en el que podrá introducir el nombre de una calle y un número de la capital de Madrid, y cuando pulse sobre un botón de "Enviar", se le mostrará una nueva página que mostrará la información de las paradas más cercanas a la calle con toda la información acerca de las líneas de autobuses que paran en esas paradas. En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial. [3 puntos]

## Normas de entrega

* Fecha tope de entrega: 22/11/2017
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre DesarrolloWeb_GrupoX donde X será el número de grupo correspondiente.

In [ ]:
# [1 punto] Crear una función que solicite al usuario dos calles de Madrid, que actúen de origen y destino de una ruta
import requests
from xml.etree import ElementTree

def pideRuta():
    
    ruta = tuple() # Variable a devolver

    origen = input("Introduzca la calle origen de la ruta: ")

    while(origen == ""):
        print("El origen no puede estar vacío. Vuelva a introducir el origen: ")
        origen = input("Introduzca la calle origen de la ruta: ")
    
    numOrigen = input("Introduzca el número del origen: ") # No comprobamos que sea distinto de "" porque puede que sea una plaza sin número

    destino = input("Introduzca la calle destino de la ruta: ")

    while(destino == ""):
        print("El destino no puede estar vacío. Vuelva a introducir el destino: ")
        destino = input("Introduzca la calle destino de la ruta: ")

    numDestino = input("Introduzca el número del destino: ") # No comprobamos que sea distinto de "" porque puede que sea una plaza sin número
        
    ruta = ((origen, numOrigen), (destino, numDestino))
    
    return ruta

# [2 puntos]Crear una función que dada una calle de Madrid, devuelva las coordenadas X e Y de la calle.

def devuelveCoordenadas(calle, numero):
    
    datos = {
    'idClient':'WEB.SERV.alvacruz@ucm.es',
    'passKey':'729F7256-3DB9-4889-8DDA-9F17887B7EC0',
    'description': str(calle),
    'streetNumber': str(numero),
    'Radius':'',
    'Stops':'',
    'statistics':'',
    'cultureInfo':''
    }
    
    url = 'https://servicios.emtmadrid.es:8443/geo/servicegeo.asmx/GetStreet'
    response = requests.post(url, data=datos) # Datos en XML
    
    # Si la calle no se encuentra, se lanza excepción y se detiene la ejecución
    
    with open("coord.xml","w") as f:
        f.write(str(response.text))
        f.close()
    
    f = open("coord.xml","rt")
    
    parseado = ElementTree.parse(f)
    
    raiz = parseado.getroot()
    
    # Conocemos la estructura del XML por lo que podemos acceder de forma indexada
    elemX = raiz[0][7].text        
    
    elemY = raiz[0][8].text
               
    coord = (elemX, elemY)
    
    return coord


def calculaRutaOptima(coordenadasOrigen, coordenadasDestino): # Ambas son tuplas con X Y
    
    datos = {
    'idClient':'WEB.SERV.pabgom03@ucm.es',
    'PassKey':'D75B3AB7-666C-4F71-A321-E265FD357B5D',
    'statistics':'',
    'cultureInfo':'',
    'coordinateXFrom': str(coordenadasOrigen[0]),
    'coordinateYFrom': str(coordenadasOrigen[1]),
    'originName':'',
    'coordinateXTo': str(coordenadasDestino[0]),
    'coordinateYTo': str(coordenadasDestino[1]),
    'destinationName':'',
    'criteriaSelection':'13',
    'day':'',
    'month':'',
    'year':'',
    'hour':'',
    'minute':'',
    'GenerarAudio':''
}
    
    url = 'https://servicios.emtmadrid.es:8443/servicemedia/servicemedia.asmx/GetStreetRoute'
    response = requests.post(url, data=datos)
    
    with open("rutaOptima.xml","w") as f:
        f.write(str(response.text))
        f.close()
    Todo = []
    f = open("rutaOptima.xml","rt")
    arbol = ElementTree.parse(f)
    root = arbol.getroot()
    for nodo in root.iter("DescriptionRouteData"):
        fecha_de_la_ruta = nodo.find("DescriptionDate").text
        hora_de_inicio_de_la_ruta = nodo.find("DescriptionInitTime").text
        hora_estimada_de_llegada = "Hora estimada de llegada: " + nodo.find("DescriptionEstimateTimeArrival").text
        numero_de_trasbordos = "Número de transbordos: " +nodo.find("Transfers").text
        duracion_del_viaje ="Duración del viaje: " + nodo.find ("LongJourney").text + " min"
        print("\nFecha del viaje: " + fecha_de_la_ruta)
        print("Hora de inicio: " + hora_de_inicio_de_la_ruta) 
        print(hora_estimada_de_llegada) 
        print(numero_de_trasbordos) 
        print(duracion_del_viaje)
        Todo += [fecha_de_la_ruta,hora_de_inicio_de_la_ruta,hora_estimada_de_llegada,numero_de_trasbordos,duracion_del_viaje]
    print("\nDescripción textual de la ruta:")
    
    # Lo recibido en el XML de la EMT muestra un "Desde  caminar" que queda incompleto al principio y un "caminar hasta" también
    # incompleto, al final, lo cual difiere de la presentación sugerida en el guión, así que truncamos la información
    # recibida para que tenga una buena presentación
    description = ""
    for nodo in root.iter("RouteDescription"):
        description += " " + nodo.text + " "
        
    tam = len(description)
    Todo += [description[8:tam - 7]] # Truncamos la info para que tenga una buena presentación
    print(Todo)
    print("\n")
    f.close()
    return Todo


# [1 punto]Crear una función que solicite al usuarios los nombres de 2 calles de Madrid y muestre la
# información referente a la ruta más óptima para llegar de una calle a otra usando autobuses de la EMT.

def infoRutaOptima ():
    
    calles = pideRuta() # calles es tupla origen-destino
    
    coordenadasOrigen = devuelveCoordenadas(calles[0][0], calles[0][1])
    coordenadasDestino = devuelveCoordenadas(calles[1][0], calles[1][1])
    
    calculaRutaOptima(coordenadasOrigen, coordenadasDestino)


def puntosInteres(coordenadasOrigen, coordenadasDestino):

    # calles = pideRuta() # calles es tupla origen-destino

    # Obtener ruta óptima
    Todo = []
    tmp = ""
    calculaRutaOptima(coordenadasOrigen, coordenadasDestino)
    
    # Ahora usamos rutaOptima.xml, que sea ha creado en calculRutaOptima
    f = open("rutaOptima.xml","rt")
    arbol = ElementTree.parse(f)
    root = arbol.getroot()
    for nodo in root.iter("POI"):
        elem = nodo.find("nombre").text
        if(elem is not None):
            elem = elem.strip()
            if elem != "" and elem != '\n': # Para evitar elementos vacíos y saltos de línea
                tmp += "Nombre del local: " + str(elem)
        
        elem = nodo.find("direccion").text
        if(elem is not None):
            elem = elem.strip()
            if elem != "" and elem != '\n': # Para evitar elementos vacíos y saltos de línea
                tmp += " Dirección del local: " + str(elem)
        
        elem = nodo.find("telefono").text
        if(elem is not None):
            elem = elem.strip()
            if elem != "" and elem != '\n': # Para evitar elementos vacíos y saltos de línea
                tmp += " Teléfono del local: " + str(elem)
        if tmp != "" and tmp != '\n':
            Todo += [tmp]
        tmp = ""
    f.close()
    return Todo

def Servicio3():
    Todo = []
    tmp = ""
    f = open("coord.xml","rt")
    arbol = ElementTree.parse(f)
    root = arbol.getroot()
    for elem in root.iter("Stop"):
        tmp += "Número de parada: " + elem.find("IdStop").text
        for nodo in elem.iter("Line"):
            tmp += " Número de línea: " + nodo.find("IdLine").text
            if nodo.find("Direction").text == "A" :
                tmp += " De " + nodo.find("HeaderA").text + " hacia " + nodo.find("HeaderB").text
            else:
                tmp += " De " + nodo.find("HeaderB").text + " hacia " + nodo.find("HeaderA").text
            tmp += " Tipo de día: " + nodo.find("DayType").text + " Hora de inicio: " + nodo.find("StartTime").text + " Último servicio: " + nodo.find("StopTime").text
            tmp += " Frecuencia mínima: " + nodo.find("MinimunFrequency").text + " Frecuencia máxima: " + nodo.find("MaximumFrequency").text

        Todo += [tmp]
        tmp = ""
    f.close()
    print(Todo)
    return Todo

In [ ]:
from bottle import route, default_app, template, run, static_file, error,request
from lxml import etree
@route('/')
def index():
    return template("index.tpl")

#opcion servicio 1
@route('/FormularioServicio1') 
def FormularioServicio1():
    return '''
        <form action="/FormularioServicio1" method="post">
            Calle de origen: <input name="calleO" type="text" />
            Número de origen: <input name="numeroO" type="text" />
            Calle de destino: <input name="calleD" type="text" />
            Número de destino: <input name="numeroD" type="text" />
            <input value="Enviar" type="submit" />
        </form>'''
@route('/FormularioServicio1',method='POST') 
def do_FormularioServicio1():
    calleO = request.forms.get('calleO')
    numeroO = request.forms.get('numeroO')
    coordO = devuelveCoordenadas(calleO, numeroO)
    calleD = request.forms.get('calleD')
    numeroD = request.forms.get('numeroD')
    coordD = devuelveCoordenadas(calleD, numeroD)
    Ruta = calculaRutaOptima(coordO, coordD)
    return template("temp.tpl",mun = Ruta)


#opcion servicio 2
@route('/FormularioServicio2') 
def FormularioServicio2():
    return '''
        <form action="/FormularioServicio2" method="post">
            Calle de origen: <input name="calleO" type="text" />
            Número de origen: <input name="numeroO" type="text" />
            Calle de destino: <input name="calleD" type="text" />
            Número de destino: <input name="numeroD" type="text" />
            <input value="Enviar" type="submit" />
        </form>'''
@route('/FormularioServicio2',method='POST') 
def do_FormularioServicio2():
    calleO = request.forms.get('calleO')
    numeroO = request.forms.get('numeroO')
    coordO = devuelveCoordenadas(calleO, numeroO)
    calleD = request.forms.get('calleD')
    numeroD = request.forms.get('numeroD')
    coordD = devuelveCoordenadas(calleD, numeroD)
    Ruta = puntosInteres(coordO, coordD)
    return template("temp.tpl",mun = Ruta)


#opcion servicio 3
@route('/FormularioServicio3') 
def FormularioServicio3():
    return '''
        <form action="/FormularioServicio3" method="post">
            Calle de Madrid: <input name="calleM" type="text" />
            Número Madrid: <input name="numeroM" type="text" />
            <input value="Enviar" type="submit" />
        </form>'''
@route('/FormularioServicio3',method='POST') 
def do_FormularioServicio3():
    calleM = request.forms.get('calleM')
    numeroM = request.forms.get('numeroM')
    devuelveCoordenadas(calleM, numeroM)
    Ruta = Servicio3()
    return template("temp.tpl",mun = Ruta)
@route('/static/<filepath:path>')
def server_static(filepath):
    return static_file(filepath, root='static')

@error(404)
def error404(error):
    return 'Error en la página solicitada'

run(host='localhost', port=8080)